### Cat vs Dog
Objective of the model is to identify Cat or Dog. 
Downlaoded the sample Data from Kaggle (https://www.kaggle.com/c/dogs-vs-cats/data). Not uploading the data as part of source code as its size is more.
Used GCP instance for analytsis. GPUs(1 x NVIDIA Tesla T4)

In [1]:
## Imports
import tensorflow as tf
from keras.datasets import mnist
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils
import os
import shutil

/home/analytics/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/analytics/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/analytics/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/analytics/anaconda3/lib/python3.6/site-packages/tensorflow/python/f

In [2]:
##limit TensorFlow GPU memory fraction: For example, the following will make sure TensorFlow uses <= 90% of your RAM:
## Also make sure you delete rm -rf ~/.nv
import keras
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
keras.backend.tensorflow_backend.set_session(tf.Session(config=config))

### Pre Process
We will move Images under cat and dog folder respectively, from their names, cat images are named as cat.XXXX.jpg and dog as dog.XXXX.jpg

In [3]:
def copy_files(prefix,start_idx,end_idx,base_dir,source_dir,target_dir):
    images_paths = [os.path.join(base_dir,source_dir,prefix+'.'+str(i)+'.jpg') for i in range(start_idx,end_idx)]
    dest_dir = os.path.join(base_dir,'model_input',target_dir,prefix)
    print(dest_dir)
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
    for image_path in images_paths:
        shutil.copy(image_path,dest_dir)
    

In [4]:
IMAGE_DIR = 'data/cat_vs_dog'

### Env.
With the above GPU also I was getting OOM error, hence reduced the sample size as below
training set 5000 of each category and test set 1000 of each
with 5000 each set we saw
0] Tesla T4         | 75'C,   0 % | 14787 / 15079 MB | python/31063(14769M)

In [11]:
## We would copy only 5000 files of each category for training and 1000 of each for testing.
copy_files('dog',0,5000,IMAGE_DIR,'train','train')
copy_files('cat',0,5000,IMAGE_DIR,'train','train')

data/cat_vs_dog/model_input/train/dog
data/cat_vs_dog/model_input/train/cat


In [5]:
copy_files('dog',5000,6000,IMAGE_DIR,'train','test')
copy_files('cat',5000,6000,IMAGE_DIR,'train','test')

data/cat_vs_dog/model_input/test/dog
data/cat_vs_dog/model_input/test/cat


In [6]:
## Constants Declarations
IMAGE_HEIGHT,IMAGE_WIDTH = 150,150
INPUT_SHAPE=(IMAGE_HEIGHT,IMAGE_WIDTH,3)
### Model parameter
BATCH_SIZE = 128
EPOCH = 10
NO_CLASSES = 2
TRAINING_SIZE = 5000
TEST_SIZE = 1000
VALIDATION_SIZE = 1000
EPOCH_STEPS = TRAINING_SIZE//BATCH_SIZE
TEST_STEP = TEST_SIZE//BATCH_SIZE

In [7]:
MODELING_INPUT_DIR = os.path.join(IMAGE_DIR,'model_input')
train_dir = os.path.join(MODELING_INPUT_DIR,'train')
test_dir = os.path.join(MODELING_INPUT_DIR,'test')

### Generator
We would use generator as Images are more, Here we would use keras Image generator

In [8]:
train_generator  = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [9]:
test_generator  = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [12]:
train_images = train_generator.flow_from_directory(train_dir,batch_size=BATCH_SIZE,target_size=(IMAGE_WIDTH,IMAGE_HEIGHT))

Found 10000 images belonging to 2 classes.


In [13]:
test_images = test_generator.flow_from_directory(test_dir,batch_size=BATCH_SIZE,target_size=(IMAGE_WIDTH,IMAGE_HEIGHT))

Found 2000 images belonging to 2 classes.


### Basic CNN Model
Let us build a simple cnn model

In [14]:
def build_cnn_model(input_shape,no_classes):
    model = Sequential()
    model.add(Conv2D(filters=64,kernel_size=(3,3), activation = 'relu', input_shape = input_shape))
    model.add(MaxPooling2D(pool_size = (2,2)))
    # Add another:
    model.add(Conv2D(filters=64,kernel_size=(3,3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))

    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(rate=0.3))
    # Add a softmax layer with 10 output units:
    model.add(Dense(no_classes, activation='sigmoid'))

    model.compile(optimizer="adam",
              loss='binary_crossentropy',
              metrics=['accuracy'])
    return model

In [15]:
cnn_model = build_cnn_model(INPUT_SHAPE,NO_CLASSES)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [16]:
cnn_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 64)      1792      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 82944)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                5308480   
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)               

In [17]:
##Let us train the CNN Model

In [18]:
cnn_model.fit_generator(train_images,steps_per_epoch=EPOCH_STEPS,epochs=EPOCH,validation_data=test_images,validation_steps=TEST_STEP)



Epoch 1/10
39/39 [==============================] - 13s 338ms/step - loss: 0.7464 - accuracy: 0.5218 - val_loss: 0.6981 - val_accuracy: 0.5402
Epoch 2/10
39/39 [==============================] - 12s 313ms/step - loss: 0.6823 - accuracy: 0.5590 - val_loss: 0.6618 - val_accuracy: 0.6473
Epoch 3/10
39/39 [==============================] - 14s 356ms/step - loss: 0.6234 - accuracy: 0.6649 - val_loss: 0.5953 - val_accuracy: 0.6415
Epoch 4/10
39/39 [==============================] - 11s 285ms/step - loss: 0.6009 - accuracy: 0.6739 - val_loss: 0.4911 - val_accuracy: 0.7171
Epoch 5/10
39/39 [==============================] - 13s 337ms/step - loss: 0.5530 - accuracy: 0.7195 - val_loss: 0.5091 - val_accuracy: 0.7087
Epoch 6/10
39/39 [==============================] - 12s 306ms/step - loss: 0.5327 - accuracy: 0.7334 - val_loss: 0.4474 - val_accuracy: 0.7355
Epoch 7/10
39/39 [==============================] - 13s 330ms/step - loss: 0.4847 - accuracy: 0.7633 - val_loss: 0.4910 - val_accuracy: 0.735